In [1]:
#pip install neo4j

In [2]:
from neo4j import GraphDatabase

In [3]:
NEO4J_URI     = 'neo4j+s://264d8780.databases.neo4j.io'
NEO4J_USERNAME= 'neo4j'
NEO4J_PASSWORD= '5l2648jhBn6kzOFi_XcK_yzYCVFzZIpoOPW7xp7M_Ss'
AURA_INSTANCEID= '264d8780'
AURA_INSTANCENAME = 'Instance01'

In [4]:
# Connect to the Neo4j AuraDB Free instance
class AuraDBConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        if self.driver:
            self.driver.close()
    
    def execute_query(self, query):
        with self.driver.session() as session:
            return session.run(query).data()

In [5]:
# Initialize connection
aura_db = AuraDBConnection(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

In [6]:
query = "MATCH (n) RETURN COUNT(n) AS node_count"

result = aura_db.execute_query(query)

print("Number of nodes in the database:", result)

Number of nodes in the database: [{'node_count': 195}]


In [7]:
# Run a sample query
query = "MATCH (n) RETURN n LIMIT 5"
result = aura_db.execute_query(query)

In [8]:
# Print results
print("Query Results:", result)

Query Results: [{'n': {'taglineEmbedding': [0.04398548603057861, -0.0010360321030020714, 0.007490075193345547, 0.07138020545244217, 0.01584959775209427, -0.01927037537097931, -0.02691010944545269, 0.016947098076343536, -0.035747114568948746, 0.028021860867738724, -0.026097673922777176, 0.01753148064017296, -0.007504328619688749, -0.017688265070319176, 0.017745278775691986, 0.010483254678547382, -0.022035501897335052, 0.01559303980320692, -0.03127159923315048, 0.0437859408557415, 0.06516579538583755, 0.05729800835251808, -0.04395698010921478, -0.014253235422074795, -0.004044355824589729, -0.024444298818707466, 0.0020845357794314623, 0.03104354813694954, 0.03378016874194145, -0.008644587360322475, 0.020909497514367104, -0.04358639568090439, 0.0061645242385566235, 0.002027522772550583, -0.021565144881606102, -0.010012897662818432, -0.012008351273834705, -0.0019063702784478664, 0.013504941016435623, -0.020082809031009674, 0.04042217507958412, 0.004190451465547085, 0.009457021951675415, 0.0

In [9]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [10]:
def add_person_and_movie(tx, person_name, person_age, movie_title, movie_year):
    query = (
        f"CREATE (p:Person {{name:  '{person_name}', age: {person_age}}}) "
        f"CREATE (m:Movie  {{title: '{movie_title}', released: {movie_year}}}) "
        f"CREATE (p)-[:LIKES]->(m)"
    )
    tx.run(query)

In [11]:
# Run the query within a session
with driver.session() as session:
    session.execute_write(add_person_and_movie, "John", 30, "Inception", 2010)

In [12]:
def add_multiple_nodes_and_relationships(tx):
    query = (
        "CREATE (p1:Person {name: 'Alice', age: 25}), "
               "(p2:Person {name: 'Bob', age: 28}), "
               "(m1:Movie {title: 'Inception', released: 2010}), "
               "(m2:Movie {title: 'The Matrix', released: 1999}), "
               "(p1)-[:LIKES]->(m1), "
               "(p2)-[:LIKES]->(m2), "
               "(p1)-[:FRIENDS_WITH]->(p2)"
    )
    tx.run(query)

In [13]:
def add_rated_relationship(tx, person_name, movie_title, rating):
    query = (
        f"MATCH (p:Person {{name: '{person_name}'}}), "
        f"(m:Movie {{title: '{movie_title}'}}) "
        f"CREATE (p)-[:RATED {{rating: {rating}}}]->(m)"
    )
    tx.run(query)

In [14]:
# Running the queries within a session
with driver.session() as session:
    session.execute_write(add_person_and_movie, "John", 30, "Inception", 2010)
    session.execute_write(add_person_and_movie, "Alice", 25, "The Matrix", 1999)
    session.execute_write(add_multiple_nodes_and_relationships)
    session.execute_write(add_rated_relationship, "Charlie", "Inception", 8.5)

In [15]:
query = "MATCH (n) RETURN COUNT(n) AS node_count"
result = driver.execute_query(query)
print("Number of nodes in the database:", result)

Number of nodes in the database: EagerResult(records=[<Record node_count=205>], summary=<neo4j._work.summary.ResultSummary object at 0x000002946B4239B0>, keys=['node_count'])


In [16]:
def view_all_nodes(tx):
    query = "MATCH (n) RETURN n LIMIT 25"
    result = tx.run(query)
    for record in result:
        print(record['n'])

In [17]:
# Run the query to view all nodes
with driver.session() as session:
    session.execute_read(view_all_nodes)

<Node element_id='4:d0d3f0af-4e3a-4953-8d37-f60e14f01827:0' labels=frozenset({'Movie'}) properties={'taglineEmbedding': [0.04398548603057861, -0.0010360321030020714, 0.007490075193345547, 0.07138020545244217, 0.01584959775209427, -0.01927037537097931, -0.02691010944545269, 0.016947098076343536, -0.035747114568948746, 0.028021860867738724, -0.026097673922777176, 0.01753148064017296, -0.007504328619688749, -0.017688265070319176, 0.017745278775691986, 0.010483254678547382, -0.022035501897335052, 0.01559303980320692, -0.03127159923315048, 0.0437859408557415, 0.06516579538583755, 0.05729800835251808, -0.04395698010921478, -0.014253235422074795, -0.004044355824589729, -0.024444298818707466, 0.0020845357794314623, 0.03104354813694954, 0.03378016874194145, -0.008644587360322475, 0.020909497514367104, -0.04358639568090439, 0.0061645242385566235, 0.002027522772550583, -0.021565144881606102, -0.010012897662818432, -0.012008351273834705, -0.0019063702784478664, 0.013504941016435623, -0.02008280903